## Updated at 17.10.2023

In [1]:
import psycopg2.extras
import psycopg2
from os import environ
import pandas as pd

postgres_config = {
    "host":environ["DB_HOST"],
    "port":environ["DB_PORT"],
    "user": environ["DB_USER"],
    "password": environ["DB_PASSWORD"],
    "database": environ["DB_NAME"],
}


## Player Selection

- Check the form of all players for the past 6 games
- Check the form of all teams for the past 6 games
- For each team define an attacking and defending rating (average xGoals against while playing at home + average xGoals while playing away, similarly for defend)

## Decision

- Attacking Players: Find the 4 teams that play against teams with bad defending rating in the next games and show their best players
- Defending Players: Find the teams with good defence overall

In [2]:
query = """
select * from team_future_opponents_ratings order by overall_opponent_defending_rating desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()
playing_against = dict(zip(df['team'], df['opponents']))
teams_selected = tuple(df.loc[:4, 'team'].to_list())

query = """
select 
    player_name,
    team,
    total_minutes_played,
    "xTotal", "xG", "xA", key_passes, shots, goals, assists,
    latest_match
from (
    select 
        row_number() over (partition by team order by "xTotal" desc) as row_num,
        *
    from player_form 
    where team in %(teams_selected)s
)
order by "xTotal" desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query, {"teams_selected":teams_selected})
    df = pd.DataFrame(pg_cur.fetchall())

df['playing_against'] = df['team'].apply(lambda x: playing_against[x])
df

,player_name,team,total_minutes_played,xTotal,xG,xA,key_passes,shots,goals,assists,latest_match,playing_against
0,Bukayo Saka,Arsenal,531.0,5.797619,4.263064,1.534555,15.0,15.0,3.0,2.0,2023-09-30,"{Chelsea,""Sheffield United"",""Newcastle United""..."
1,Ollie Watkins,Aston Villa,526.0,5.364430,3.197336,2.167095,11.0,22.0,4.0,3.0,2023-10-08,"{""West Ham"",Luton,""Nottingham Forest"",Fulham,T..."
2,Moussa Diaby,Aston Villa,489.0,3.583459,1.101973,2.481486,8.0,12.0,1.0,3.0,2023-10-08,"{""West Ham"",Luton,""Nottingham Forest"",Fulham,T..."
3,Matthew Cash,Aston Villa,536.0,2.786229,2.214701,0.571528,1.0,14.0,2.0,1.0,2023-10-08,"{""West Ham"",Luton,""Nottingham Forest"",Fulham,T..."
4,Taiwo Awoniyi,Nottingham Forest,493.0,2.757451,2.081895,0.675556,9.0,8.0,2.0,2.0,2023-10-01,"{Luton,Liverpool,""Aston Villa"",""West Ham"",Brig..."
5,Martin Odegaard,Arsenal,536.0,2.602663,1.466473,1.136190,11.0,15.0,2.0,1.0,2023-10-08,"{Chelsea,""Sheffield United"",""Newcastle United""..."
6,Morgan Gibbs-White,Nottingham Forest,477.0,2.367930,0.918247,1.449683,10.0,11.0,0.0,1.0,2023-10-07,"{Luton,Liverpool,""Aston Villa"",""West Ham"",Brig..."
7,Leon Bailey,Aston Villa,225.0,2.289843,1.301388,0.988455,4.0,4.0,2.0,1.0,2023-10-08,"{""West Ham"",Luton,""Nottingham Forest"",Fulham,T..."
8,Lyle Foster,Burnley,528.0,2.193729,1.944564,0.249165,4.0,10.0,3.0,2.0,2023-10-07,"{Brentford,Bournemouth,""Crystal Palace"",Arsena..."
9,Zeki Amdouni,Burnley,429.0,1.813232,1.281716,0.531515,4.0,12.0,1.0,0.0,2023-10-03,"{Brentford,Bournemouth,""Crystal Palace"",Arsena..."


## The best players based on their recent form (past 6 games)

In [3]:

query = """
select
    player_name,
    team,
    total_minutes_played,
    "xTotal", "xG", "xA", key_passes, shots, goals, assists,
    latest_match, team_played_against
from player_form
order by "xTotal" desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()
df.head(20)

,player_name,team,total_minutes_played,xTotal,xG,xA,key_passes,shots,goals,assists,latest_match,team_played_against
0,Erling Haaland,Manchester City,540.0,8.698612,7.370574,1.328038,7.0,24.0,6.0,1.0,2023-10-08,"{Arsenal,""Wolverhampton Wanderers"",""Nottingham..."
1,Mohamed Salah,Liverpool,531.0,7.823171,3.556305,4.266867,14.0,18.0,4.0,3.0,2023-10-08,"{Brighton,Tottenham,""West Ham"",""Wolverhampton ..."
2,Bukayo Saka,Arsenal,531.0,5.797619,4.263064,1.534555,15.0,15.0,3.0,2.0,2023-09-30,"{Bournemouth,Tottenham,Everton,""Manchester Uni..."
3,Ollie Watkins,Aston Villa,526.0,5.364430,3.197336,2.167095,11.0,22.0,4.0,3.0,2023-10-08,"{""Wolverhampton Wanderers"",Brighton,Chelsea,""C..."
4,Son Heung-Min,Tottenham,489.0,5.157540,3.452923,1.704617,10.0,20.0,6.0,0.0,2023-10-07,"{Luton,Liverpool,Arsenal,""Sheffield United"",Bu..."
5,Alexander Isak,Newcastle United,413.0,5.080034,4.914274,0.165760,3.0,15.0,4.0,0.0,2023-10-08,"{""West Ham"",Burnley,""Sheffield United"",Brighto..."
6,Abdoulaye Doucouré,Everton,510.0,5.007516,4.284884,0.722632,11.0,13.0,3.0,0.0,2023-10-07,"{Bournemouth,Luton,Brentford,Arsenal,""Sheffiel..."
7,Darwin Núñez,Liverpool,286.0,4.920577,3.691971,1.228606,3.0,15.0,3.0,2.0,2023-10-08,"{Brighton,""West Ham"",""Wolverhampton Wanderers""..."
8,Nicolas Jackson,Chelsea,466.0,4.569864,3.914147,0.655716,5.0,16.0,2.0,0.0,2023-10-07,"{Burnley,""Aston Villa"",Bournemouth,""Nottingham..."
9,Bruno Fernandes,Manchester United,540.0,4.199396,2.006766,2.192630,15.0,18.0,2.0,1.0,2023-10-07,"{Brentford,""Crystal Palace"",Burnley,Brighton,A..."


## The recent team form (past 6 games) ordered by the best defence lately to the worst one

In [4]:

query = """
select
    *
from team_form
order by defending_rating asc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()
df

,team,xG,xGA,xpts,attack_home,attack_away,defence_home,defence_away,matches_home,matches_away,avg_home_att,avg_away_att,avg_home_def,avg_away_def,defending_rating,attacking_rating
0,Arsenal,12.746873,5.813270,11.9188,7.870068,4.876805,4.885657,0.927613,4.0,2.0,1.967517,2.438403,1.221414,0.463807,1.685221,4.405919
1,Newcastle United,14.198400,5.124965,13.0855,6.206790,7.991610,2.291648,2.833317,3.0,3.0,2.068930,2.663870,0.763883,0.944439,1.708322,4.732800
2,Manchester City,13.178681,4.911786,12.9726,3.341360,9.837321,2.193379,2.718407,2.0,4.0,1.670680,2.459330,1.096690,0.679602,1.776291,4.130010
3,Chelsea,12.252810,5.922206,12.3186,6.895770,5.357040,2.751031,3.171175,3.0,3.0,2.298590,1.785680,0.917010,1.057058,1.974069,4.084270
4,Tottenham,13.597450,7.254285,12.6988,4.347320,9.250130,1.913645,5.340640,2.0,4.0,2.173660,2.312533,0.956822,1.335160,2.291983,4.486192
5,Crystal Palace,7.051882,7.720109,7.5292,3.684162,3.367720,2.902838,4.817271,3.0,3.0,1.228054,1.122573,0.967613,1.605757,2.573370,2.350627
6,Manchester United,8.402003,7.959848,8.2976,6.595621,1.806382,5.114674,2.845174,4.0,2.0,1.648905,0.903191,1.278669,1.422587,2.701256,2.552096
7,Liverpool,14.190411,8.733586,11.2494,6.874000,7.316411,2.154283,6.579303,2.0,4.0,3.437000,1.829103,1.077141,1.644826,2.721967,5.266103
8,Everton,14.253360,7.845837,12.1032,8.999680,5.253680,4.696817,3.149020,4.0,2.0,2.249920,2.626840,1.174204,1.574510,2.748714,4.876760
9,Aston Villa,10.626023,9.166615,9.3903,4.222640,6.403383,2.845150,6.321465,2.0,4.0,2.111320,1.600846,1.422575,1.580366,3.002941,3.712166


## Show players with good overall stats

In [5]:

query = """
select
    id, player_name, games, time, goals, assists,
    "xG", "xA", "xG" + "xA" as "xTotal", 
    ("xG" + "xA") / games as "xTotal_per_game", 
    "xGChain" / games as "xGChain_per_game"
from player
where games > 3
order by "xTotal_per_game" desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()
df.head(30)

,id,player_name,games,time,goals,assists,xG,xA,xTotal,xTotal_per_game,xGChain_per_game
0,1250,Mohamed Salah,8,705,5,4,5.663196,4.713721,10.376917,1.297115,0.963652
1,8260,Erling Haaland,8,715,8,1,8.660618,1.668504,10.329123,1.291140,0.986907
2,5232,Alexander Isak,7,492,6,0,6.077839,0.165760,6.243600,0.891943,0.787984
3,7322,Bukayo Saka,7,621,4,2,4.498355,1.683985,6.182341,0.883192,0.705725
4,1726,Abdoulaye Doucouré,8,690,3,0,4.871711,1.529557,6.401268,0.800159,0.638816
5,8865,Ollie Watkins,8,699,4,4,3.983381,2.399880,6.383261,0.797908,0.857299
6,10048,Nicolas Jackson,7,556,2,0,4.758097,0.655716,5.413813,0.773402,0.858655
7,453,Son Heung-Min,8,665,6,0,3.727889,2.356697,6.084586,0.760573,0.982037
8,1228,Bruno Fernandes,8,720,2,1,2.797997,3.245947,6.043944,0.755493,0.665284
9,10720,Darwin Núñez,7,303,3,2,3.730780,1.228606,4.959386,0.708484,0.733646
